# Text Generation with `transformer` models 

Return to the [castle](https://github.com/Nkluge-correa/teeny-tiny_castle).

**`Text generation` models are a type of machine learning model that can generate natural language text. These models have a wide range of applications, including `language translation`, `summarization`, and `content creation`. There are several different approaches to text generation, including `statistical models`, `rule-based systems`, and more recently, `neural network-based models`. These models can be trained on large amounts of input text data and use this information to generate new text that is coherent and reflects the patterns and structures found in the training data. The quality of the generated text can vary depending on the _complexity of the model and the amount and quality of the training data_. Overall, text-generation models have the potential to revolutionize many industries by automating the creation of written content and enabling more efficient communication.**

**One of the biggest and most current advances in language modeling was made possible by the invention of the `transformer` architecture. A `transformer` model is a type of neural network architecture first described in the 2017 paper "_[Attention Is All You Need](https://arxiv.org/abs/1706.03762)_" by Vaswani et al. Transformers were originally used for machine translation, but this adaptable architecture is now used in a wide range of fields and problems.**

<img src="https://machinelearningmastery.com/wp-content/uploads/2021/08/attention_research_1.png" alt="drawing" height="450"/>

**Before the `transformer`, most language models used recurrent neural networks ([`RNNs`](https://en.wikipedia.org/wiki/Recurrent_neural_network)) to process language data. While effective, `RNNs` can be slow and difficult to parallelize, limiting their ability to scale to large datasets. The `transformer` architecture was designed to overcome these limitations and allow for the efficient processing of long sequences of data. It does this by using `self-attention` mechanisms, which allow the model to weigh different input elements differently and to consider the relationships between all elements in the input sequence simultaneously. This allows the `transformer` to process input data in parallel rather than sequentially like an `RNN`.**

**The `transformer` architecture has enabled the development of powerful language models such as [`BERT`](https://huggingface.co/docs/transformers/model_doc/bert) and [`GPT-3`](https://arxiv.org/abs/2005.14165), which have achieved state-of-the-art results on a wide range of natural language processing tasks.** 

**To learn more about the original `transformer` architecture, go to our [`sequence-to-sequence` machine translation](https://github.com/Nkluge-correa/teeny-tiny_castle/blob/a5a51e46dc0e04cb4cb795503f14200374f62e52/ML%20Intro%20Course/seuqnece-to-sequence.ipynb) notebook. To learn more about the implementetion of RNN in language modeling aplications, go to this [notebook](https://github.com/Nkluge-correa/teeny-tiny_castle/blob/a5a51e46dc0e04cb4cb795503f14200374f62e52/ML%20Explainability/NLP%20Interpreter%20(en)/model_maker_en.ipynb).**

**In this notebook, we will be implementing a `decoder-only transformer` for text generation. `Decoder-only transformers`, such as the `GPT` (_Generative Pre-training Transformer_) series, are models that consist only of a `decoder`, which takes an input sequence and generates an output sequence. `Decoder-only transformers` have been used to generate coherent text, including entire articles and stories. One advantage of decoder-only transformers is that they are relatively simple and easy to train, as they do not require an `encoder` component.** 

**To start building our model, we first need _good-quality text_. Good-quality text is important for the training of language models for several reasons. For example, good-quality text is more likely to reflect the real-world patterns and structures of language, which is important for the model to learn. If a model is trained on poorly written or grammatically incorrect text, it may struggle to generate correct and coherent output. Also, a model trained on large amounts of high-quality text may be able to learn language patterns more quickly and with fewer resources than a model trained on low-quality text.**

**For this, we will create a text dataset using the articles from the [`Stanford Encyclopedia of Philosophy`](https://plato.stanford.edu/). Details on the created text corpus can be found on this [markdown file](SEP\Stanford_Encyclopedia_of_Philosophy_Text_Corpus_v0.1.md), and the full dataset can be downloaded in [this link](https://drive.google.com/uc?export=download&id=167nlUxmY0Qc9_wGMuyMqgh2pUwyJDrte).**

## Getting a Text Corpus

**Web scraping is the process of extracting information from websites. This can be done using a variety of programming languages and tools, such as Python and its libraries for web scraping, such as `BeautifulSoup` and `Scrapy`.**

**Web scraping can be useful but also can be done in an unethical way and even illegally, so it is important to be aware of the website's terms of use before scraping. Many websites have terms of service that prohibit scraping, so it's important to review the terms of service of a website before scraping it.**

**If we check the [`robots.txt`](https://plato.stanford.edu/robots.txt) file of the SEP, we see what we are allowed to do, and the following code will only scrape permissible content of the SEP.**

**We will be using `BeautifulSoup` to get our text data in this example.**

In [ ]:
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup


# Set the URL to scrape
url = "https://plato.stanford.edu/contents.html"

# Send a request to the URL and retrieve the webpage
page = requests.get(url)

# Parse the webpage with BeautifulSoup
soup = BeautifulSoup(page.text, "html.parser")

# parse the href addresses and anchors of the html page
definitions = soup.find_all("a")
quoted = re.compile('"[^"]*"')

entries = []
for definition in definitions:
    definition = str(definition)

    # get all the addresses of the links in the 'contents.html' page
    for value in quoted.findall(definition):

        # get all pages, that have philosophical text
        if value[1:-1].startswith("entries"):
            entries.append(value[1:-1])

# list of all the texts
paragraphs = []

# list of all the source pages
source = []

**Now that we have all the entries, we can begin to extract the text from them (note that the entries are allowed according to the [`robots.txt`](https://plato.stanford.edu/robots.txt)). ALWAYS BE SURE YOU ARE ALLOWED TO SCRAPE.**

In [2]:
print(f'Number of content pages: {len(entries)}')
# loop over all the pages that have philosophical text
for i, entri in enumerate(entries):
    url = f"https://plato.stanford.edu/{entri}"

    # Send a request to the URL and retrieve the webpage
    page = requests.get(url)

    # Parse the webpage with BeautifulSoup
    soup = BeautifulSoup(page.text, "html.parser")

    # Get all the <p> tags from the parsed html
    texts = soup.find_all("p")

    # Loop over all the <p> tags from the parsed html
    for text in texts:

        # remove the html tags from the string
        clean_string = re.sub(r'<[^>]*>', '', str(text))

        # replace the '\n' with " "
        clean_string = clean_string.replace("\n", " ")

        # append the source and text elements
        paragraphs.append(clean_string)
        source.append(url)

    print(f"Page {i + 1}. Scrapped page '{url}'!")

Number of content pages: 2385
Page 1. Scrapped page 'https://plato.stanford.edu/entries/abduction/'!
Page 2. Scrapped page 'https://plato.stanford.edu/entries/abelard/'!
Page 3. Scrapped page 'https://plato.stanford.edu/entries/abhidharma/'!
Page 4. Scrapped page 'https://plato.stanford.edu/entries/abilities/'!
Page 5. Scrapped page 'https://plato.stanford.edu/entries/abner-burgos/'!
Page 6. Scrapped page 'https://plato.stanford.edu/entries/abrabanel/'!
Page 7. Scrapped page 'https://plato.stanford.edu/entries/abstract-objects/'!
Page 8. Scrapped page 'https://plato.stanford.edu/entries/essential-accidental/'!
Page 9. Scrapped page 'https://plato.stanford.edu/entries/action/'!
Page 10. Scrapped page 'https://plato.stanford.edu/entries/shared-agency/'!
Page 11. Scrapped page 'https://plato.stanford.edu/entries/logic-action/'!
Page 12. Scrapped page 'https://plato.stanford.edu/entries/action-perception/'!
Page 13. Scrapped page 'https://plato.stanford.edu/entries/qm-action-distance/'!
Pa

**Now, let us create a dataframe with all of our text corpus.**

In [3]:
# create a pandas data frame with the data
df = pd.DataFrame({'URL': source, 'Text': paragraphs})

# drop duplicate text
df = df.drop_duplicates()

# Clean the URL to get the "Category" of the page
def clean_url(string):
    return string.split('entries/')[1][:-1]

# Apply the function to the "URL" column
df['Category'] = df['URL'].apply(clean_url)

# save as a csv file
df.to_csv('SEP\stanford_encyclopedia_philosophy.csv', index=False)

**And here is our text corpus. You can download it directly using [this link](https://drive.google.com/uc?export=download&id=167nlUxmY0Qc9_wGMuyMqgh2pUwyJDrte).**

In [9]:
import pandas as pd 

df = pd.read_csv('SEP\stanford_encyclopedia_philosophy.csv')
display(df)

,URL,Text,Category
0,https://plato.stanford.edu/entries/abduction/,"In the philosophical literature, the term “ab...",abduction
1,https://plato.stanford.edu/entries/abduction/,This entry is exclusively concerned with abdu...,abduction
2,https://plato.stanford.edu/entries/abduction/,"See also the entry on scientific discovery, ...",abduction
3,https://plato.stanford.edu/entries/abduction/,Most philosophers agree that abduction (in th...,abduction
4,https://plato.stanford.edu/entries/abduction/,You happen to know that Tim and Harry have re...,abduction
...,...,...,...
182526,https://plato.stanford.edu/entries/zombies/,Many thanks to David Chalmers and to Bill Fis...,zombies
182527,https://plato.stanford.edu/entries/zombies/,Copyright © 2019 by Robert Kirk &lt;Robert....,zombies
182528,https://plato.stanford.edu/entries/zombies/,View this site from another server:,zombies
182529,https://plato.stanford.edu/entries/zombies/,The Stanford Encyclopedia of Philosophy is cop...,zombies


**The loop below will create a dataset folder. The folder will contain a folder for each topic in the SEP. If you downloaded the dataset, you can skip the next cell.**

In [10]:
import os

for category in df.Category.unique():
    os.mkdir(f"SEP\dataset\\{category}")
    dff = df[df['Category'] == category]

    for i, sample in enumerate(list(dff.Text)):
        with open(f'SEP\dataset\\{category}\\{i}.txt', 'w', encoding='utf-8') as fp:
            fp.write(sample)

print('Dataset Folder Created!') 

Dataset Folder Created!


**Using the entire SEP to train a language model If you don't have access to powerful GPUs, Corpus can take a long time. As a result, for demonstration purposes, we will train our language model on a subset of our corpus. The same can be said for the entire dataset. However, as the size of the dataset we feed to the model grows, so will the number of training epochs, vocabulary, and model size.**

**Our mini-dataset contains only `aesthetics-18th-british`, `aesthetics-18th-french`, `aesthetics-18th-german`, `aesthetics-19th-romantic`.**

In [1]:
import os

filenames = []
directories = ["SEP\dataset\\aesthetics-18th-british",
                "SEP\dataset\\aesthetics-18th-french",
                "SEP\dataset\\aesthetics-18th-german",
                "SEP\dataset\\aesthetics-19th-romantic"]

for directory in directories:
    for folder in os.listdir(directory):
        filenames.append(os.path.join(directory, folder))

print(f"Found {len(filenames)} files")

Found 659 files


**All of the found files is a `txt` file with some text about the topics selected above.**

**Now, let us shuffle the order of our samples, and create a dataset using the `tf.data.TextLineDataset`, which loads text from text files and creates a dataset where each line of the files becomes an element of the dataset. We also selected a small `batch_size` to avoid OOM (OUT-OF-MEMORY) problems with our GPU (4GB of VRAM)**

In [2]:
import random
import tensorflow as tf

batch_size = 16

random.shuffle(filenames)

text_ds = tf.data.TextLineDataset(filenames)
text_ds = text_ds.shuffle(buffer_size=256)
text_ds = text_ds.batch(batch_size)

**Because we are using a small dataset, we will have a small vocabulary (in this example, we end up with a vocabulary that has 8600 unique tokens). This is a very small vocabulary when compared to large language models like GPT-3.**

**We create our vocabulary using the `tf.keras.layers.TextVectorization`, passing a `custom_standardization` function to lower strings and parse punctuations. Then we adapt the `TextVectorization` layer to our dataset and get our vocabulary out of it. We are saving the vocabulary in a `txt` file for later use. Is from this vocabulary that we can detokenize the sequences produced by our language model.**

In [3]:
import string

# Will cut sequences with more than 500 tokens
sequence_length = 500

# Maximum vocabulary size
vocab_size = 8700 

# Lower all strings and parse punctuation
def custom_standardization(input_string):
    lowercased = tf.strings.lower(input_string)
    return tf.strings.regex_replace(lowercased, f"([{string.punctuation}])", r" \1")

from keras.layers import TextVectorization

# Create a vectorization layer and adapt it to the text
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size - 1,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
)

# Fit the TextVectorization layer to the dataset
vectorize_layer.adapt(text_ds)

# Get words back from token indices
vocab = vectorize_layer.get_vocabulary()  

print(f'Found {len(vocab)} unique tokens!')

# Save the vocabulary as a text file
with open(f'vocabulary.txt', 'w', encoding='utf-8') as fp:
    for word in vocab:
        fp.write("%s\n" % word)

# Index to detokenize tokens
vocab_index = {}
for index, word in enumerate(vocab):
    vocab_index[word] = index

Found 8600 unique tokens!


**To get our dataset ready, we shift word sequences by $1$ position so that the target for the position ($i$) is a word at position ($i+1$). The model will use all words up till position ($i$) to predict the next word. Thus, our language model will generate text in an autoregressive fashion.**

In [4]:
def prepare_lm_dataset(text):
    text = tf.expand_dims(text, -1)
    tokenized_sentences = vectorize_layer(text)
    x = tokenized_sentences[:, :-1]
    y = tokenized_sentences[:, 1:]
    return x, y


text_ds = text_ds.map(prepare_lm_dataset)
text_ds = text_ds.prefetch(tf.data.AUTOTUNE)

print('Dataset ready!')

Dataset ready!


**In this notebook, we will create a decoder-only transformer based on the GPT architecture. Our model will use some of the same components we used in our [`sequence-to-sequence`](https://github.com/Nkluge-correa/teeny-tiny_castle/blob/a5a51e46dc0e04cb4cb795503f14200374f62e52/ML%20Intro%20Course/seuqnece-to-sequence.ipynb), like the `PositionalEmbedding` layer (a way to inject word-order information into our model), and the `TransformerDecoder` block, how is in charge with the actual prediction of the next token in a given sequence.**

**The combination of these simple blocks gives rise to our mini-GPT. If you wish to create a more robust model and give it more data, you could stack more `TransformerDecoder` blocks and create residual connections among them.**

In [5]:
from tensorflow import keras
from keras import layers

class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions
    
    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
          num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
          num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm_2 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm_3 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(0.1)
        self.dropout2 = layers.Dropout(0.1)
        self.dropout3 = layers.Dropout(0.1)
        self.supports_masking = True

    def get_config(self):
        config = super(TransformerDecoder, self).get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        attention_output_1 = self.attention_1(inputs, inputs, attention_mask=causal_mask)
        attention_output_1 = self.dropout1(attention_output_1)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(query=attention_output_1,
                                                value=encoder_outputs,
                                                key=encoder_outputs,
                                                attention_mask=padding_mask)
        attention_output_2 = self.dropout2(attention_output_2)
        attention_output_2 = self.layernorm_2(attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        proj_output = self.dropout3(proj_output)
        return self.layernorm_3(attention_output_2 + proj_output)

**We end up with a 6.6M parameters model, which is small for a modern-day, state-of-the-art language model. Keeping things simple, this model has only two attention heads, one decoder block, a vocabulary (_targets of the final dense network_) of 8600 tokens, embedding dimensions of 256 (GPT-3-Davinci has 12288 dimensions in its embedding space), and a dense-latent-dimension space (nº of neurons in the feed-forward network) of 2048.**

In [6]:
from tensorflow import keras
import keras_nlp

sequence_length = 500
embed_dim = 256
latent_dim = 2048
num_heads = 2

inputs = keras.Input(shape=(sequence_length,), dtype=tf.int32)
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, x)
outputs = layers.Dense(len(vocab), activation="softmax")(x)
model = keras.Model(inputs, outputs=outputs)

perplexity = keras_nlp.metrics.Perplexity(name="perplexity")

model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
                optimizer="adam", metrics=perplexity)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 500)]        0           []                               
                                                                                                  
 positional_embedding (Position  (None, 500, 256)    2355200     ['input_1[0][0]']                
 alEmbedding)                                                                                     
                                                                                                  
 transformer_decoder (Transform  (None, 500, 256)    2104576     ['positional_embedding[0][0]',   
 erDecoder)                                                       'positional_embedding[0][0]']   
                                                                                              

**Now, the last thing we need is to train our model. We are using `keras.callbacks` to save the best model in 30 epochs (for such a small dataset we don't need more). The callback will be monitoring the `perplexity` of the model, which is the metric we are using to evaluate performance.**

**Perplexity is a metric that measures how well a probabilistic model (such as a language model) predicts a sample. Perplexity is defined as 2 to the power of the cross-entropy, which is a measure of the difference between the sample's predicted and true probability distribution.** 

**The formula for perplexity is simply the inverse of the exponentiation of the cross-entropy:**

$$Perplexity = 2^{-cross\;entropy}$$

$$Cross\;Entropy = -\frac{1}{n}\sum_{i=1}^{n}\log P(w_i)$$

**where $n$ is the length of the sample, $P(w_i)$ is the predicted probability of the ith word in the sample according to the language model, and the sum is taken over all words in the sample.**

**Perplexity reveals how well a model predicts the next word in a sequence, and thus how well it knows the language. In general, it is used as an evaluation metric in Language Modeling.**

In [7]:
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

callbacks = [keras.callbacks.ModelCheckpoint("text_gen.keras",
                                                save_best_only=True,
                                                monitor="perplexity",
                                                patience=3, 
                                                restore_best_weights=True)]

model.fit(text_ds, verbose=1, epochs=30, callbacks=callbacks)

Version:  2.10.1
Eager mode:  True
GPU is available
Epoch 1/30


c:\Users\CWLINK\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\backend.py:5582: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


42/42 [==============================] - 26s 548ms/step - loss: 1.8061 - perplexity: 1073.2589
Epoch 2/30
42/42 [==============================] - 23s 552ms/step - loss: 1.5625 - perplexity: 441.3623
Epoch 3/30
42/42 [==============================] - 23s 555ms/step - loss: 1.3664 - perplexity: 204.2755
Epoch 4/30
42/42 [==============================] - 23s 554ms/step - loss: 1.2208 - perplexity: 116.2645
Epoch 5/30
42/42 [==============================] - 23s 553ms/step - loss: 1.1078 - perplexity: 75.1609
Epoch 6/30
42/42 [==============================] - 23s 556ms/step - loss: 1.0033 - perplexity: 49.9031
Epoch 7/30
42/42 [==============================] - 23s 554ms/step - loss: 0.8976 - perplexity: 33.0584
Epoch 8/30
42/42 [==============================] - 23s 553ms/step - loss: 0.7942 - perplexity: 22.1419
Epoch 9/30
42/42 [==============================] - 23s 552ms/step - loss: 0.7069 - perplexity: 15.7344
Epoch 10/30
42/42 [==============================] - 23s 553ms/step - 

**Having a trained language model on texts related to aesthetic philosophy, we can now use it to generate some text.**

**To load models created using subclass functions from the Keras API, you need to pass your classes as custom objects (after building the classes ...):**

```python

model = keras.models.load_model("your_model.keras",
                    custom_objects={"TransformerDecoder": TransformerDecoder, 
                                "PositionalEmbedding": PositionalEmbedding,
                                "perplexity": keras_nlp.metrics.Perplexity})

```

In [8]:
from keras.layers import TextVectorization
from tensorflow import keras
import tensorflow as tf
from keras import layers
import numpy as np
import keras_nlp

TextGenerator = keras.models.load_model("text_gen.keras", 
    custom_objects={"PositionalEmbedding": PositionalEmbedding,
        "TransformerDecoder": TransformerDecoder,
        "perplexity": keras_nlp.metrics.Perplexity})

**We also load our vocabulary to create a vocabulary index to detokenize the outputs of our model. With this vocabulary, we create a TextVectorization layer to tokenize our prompt inputs, passing our vocabulary so we don't have to adapt it again.**

In [9]:
with open('vocabulary.txt', encoding='utf-8') as file_in:
    vocab = [line.strip() for line in file_in]

vocab_index = {}
for index, word in enumerate(vocab):
    vocab_index[word] = index

text_vectorization = TextVectorization(max_tokens=len(vocab),
                                        output_mode="int",
                                        output_sequence_length=500,
                                        vocabulary=vocab)

In [10]:

# This function allows us to sample from the probability distribution output of our model with 
# a "chose_from = 1", the model will always argmax. But with a value higher than 1, 
# this function will sample a token, randomly from the top n (chose_from = n) 
# of the distribution

def sample_from(logits, chose_from):
    logits, indices = tf.math.top_k(logits, k=chose_from, sorted=True)
    indices = np.asarray(indices).astype("int32")
    preds = keras.activations.softmax(tf.expand_dims(logits, 0))[0]
    preds = np.asarray(preds).astype("float32")
    return np.random.choice(indices, p=preds)

# Function to turn tokens back into words ...

def detokenize(number):
    return vocab[number]

# This function takes as input a sequence of "start_tokens" (a prompt), a number of tokens to 
# be generated, a vocabulary, and the "chose_from" parameter. The function will output a 
# sequence of words, generated by the model, using the sampling method established. You 
# can change the number of the "chose_from" argument to make the output less 
# repetitive and more random. 

def generate_text(start_tokens, generate_tokens, vobab, chose_from):
    start_tokens = [_ for _ in start_tokens]
    num_tokens_generated = 0
    tokens_generated = []

    while num_tokens_generated <= generate_tokens:

        pad_len = sequence_length - len(start_tokens)
        sample_index = len(start_tokens) - 1
        if pad_len < 0:
            x = start_tokens[:sequence_length]
            sample_index = sequence_length - 1
        elif pad_len > 0:
            x = start_tokens + [0] * pad_len
        else:
            x = start_tokens

        x = np.array([x])
        y = TextGenerator.predict(x, verbose=0)
        sample_token = sample_from(y[0][sample_index], chose_from)
        tokens_generated.append(sample_token)
        start_tokens.append(sample_token)
        num_tokens_generated = len(tokens_generated)

    return " ".join([detokenize(_) for _ in start_tokens + tokens_generated])

**Now, let us generate some text using this seed, "_debates about artistic matters..._". We will generate 50 tokens after the last word, with a greedy sampling policy (argmaxing always).**

In [11]:
start_prompt = "debates about artistic matters"
start_tokens = [vocab_index.get(_, 1) for _ in start_prompt.split()]
generate_tokens = 50

generate_text(start_tokens, generate_tokens, vocab, 1)

'debates about artistic matters were greatly influenced by the new spaces and means of communication that emerged in the seventeenth and eighteenth centuries . critics expressed their judgments in published treatises and in periodicals such as le mercure galant ; philosophical ideas were also developed in oral conversations between members of the newly founded royal were greatly influenced by the new spaces and means of communication that emerged in the seventeenth and eighteenth centuries . critics expressed their judgments in published treatises and in periodicals such as le mercure galant ; philosophical ideas were also developed in oral conversations between members of the newly founded royal'

**As expected, the model knows "something about 17th-century aesthetics". However, a greedy sampling policy makes the output repetitive. Let us introduce some randomness.**

In [12]:
start_prompt = "debates about artistic matters"
start_tokens = [vocab_index.get(_, 1) for _ in start_prompt.split()]
generate_tokens = 50

generate_text(start_tokens, generate_tokens, vocab, 3)

'debates about artistic matters were greatly else that lessing’s work the aftermath rules . their conviction as in interpretation of the romantics never gave these arts during the ancien published treatises and the romantic texts . in 1719 ] atomistically like flirting finally a vital approach to the romantics aspired rather to cost him his were greatly else that lessing’s work the aftermath rules . their conviction as in interpretation of the romantics never gave these arts during the ancien published treatises and the romantic texts . in 1719 ] atomistically like flirting finally a vital approach to the romantics aspired rather to cost him his'

**The text appears to be meaningless at this point. This is to be expected given that we're dealing with a small model trained on even less text.**

**To summarize, developing a language model is a complex task that requires knowledge of natural language processing, machine learning, and deep learning techniques. We covered the fundamentals of language modeling with the GPT architecture and TensorFlow in this notebook. Remember that developing a good language model requires a large amount of data and computational resources, so don't be discouraged if your first attempts don't yield cutting-edge results.** 🤖

---

Return to the [castle](https://github.com/Nkluge-correa/teeny-tiny_castle).